# Transfer Learning

In [1]:
import numpy as np
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
% matplotlib inline
from keras.datasets import fashion_mnist

/Users/amitkaps/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def fashion_mnist_preprocess():
    
    num_classes = 10
    
    # get data
    (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
    x_train.shape, y_train.shape, x_test.shape, y_test.shape
    
    # input image dimensions
    img_rows, img_cols = 28, 28
    
    # reshape to channel_last format
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    
    # normalize data
    x_train_normalize = x_train / 255
    x_test_normalize = x_test / 255
    
    # convert class vectors to binary class matrices
    y_train_class = keras.utils.to_categorical(y_train, num_classes)
    y_test_class = keras.utils.to_categorical(y_test, num_classes)
    
    return (x_train_normalize, y_train_class), (x_test_normalize, y_test_class)

In [3]:
(x_train_conv, y_train_class), (x_test_conv, y_test_class) = fashion_mnist_preprocess()

## Basic CNN

In [4]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

In [5]:
cnn = Sequential()
cnn.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28,28,1)))
cnn.add(MaxPooling2D(pool_size=(2, 2)))
cnn.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2)))
cnn.add(Dropout(0.25))
cnn.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
cnn.add(Dropout(0.25))
cnn.add(Flatten())
cnn.add(Dense(128, activation='relu'))
cnn.add(Dropout(0.25))
cnn.add(Dense(10, activation='softmax'))

In [6]:
cnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 128)         73856     
_________________________________________________________________
dropout_2 (Dropout)          (None, 3, 3, 128)         0         
__________

In [7]:
cnn.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [8]:
%time
history = cnn.fit(x_train_conv, y_train_class, batch_size=128, epochs=5, verbose=1, validation_split = 0.2)

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 31.9 µs
Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 75s 2ms/step - loss: 0.6857 - acc: 0.7472 - val_loss: 0.4224 - val_acc: 0.8441
Epoch 2/5
48000/48000 [==============================] - 73s 2ms/step - loss: 0.4289 - acc: 0.8423 - val_loss: 0.3493 - val_acc: 0.8728
Epoch 3/5
48000/48000 [==============================] - 76s 2ms/step - loss: 0.3636 - acc: 0.8677 - val_loss: 0.3131 - val_acc: 0.8862
Epoch 4/5
48000/48000 [==============================] - 75s 2ms/step - loss: 0.3287 - acc: 0.8798 - val_loss: 0.2944 - val_acc: 0.8919
Epoch 5/5
48000/48000 [==============================] - 94s 2ms/step - loss: 0.3058 - acc: 0.8891 - val_loss: 0.2811 - val_acc: 0.8951


In [ ]:
def plot_metric(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(1, len(acc) + 1)

    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.ylim(0,1)
    plt.legend()

    plt.figure()

    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.ylim(0,1)
    plt.legend()

    plt.show()

In [ ]:
score = cnn.evaluate(x_test_conv, y_test_class, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

## Data Augementation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train_conv, y_train_class, test_size=0.2, random_state=12345)

In [ ]:
# this will do preprocessing and realtime data augmentation
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    rotation_range=25,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=False,  # randomly flip images
    vertical_flip=False)  # randomly flip images

In [ ]:
datagen.fit(x_train)

In [ ]:
# fits the model on batches with real-time data augmentation:
#history = cnn.fit_generator(datagen.flow(x_train_conv, y_train_class, batch_size=32), 
                            validation_data=(x_val, y_val), use_multiprocessing=True,
                    steps_per_epoch=len(x_train_conv) / 32, epochs=2)

In [ ]:
plot_metric(history)

In [ ]:
score = cnn.evaluate(x_test_conv, y_test_class, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

## Transfer Learning

In [ ]:
from keras.applications import inception_v3

In [ ]:
conv_base = inception_v3.InceptionV3(include_top=False, weights='imagenet', 
                        input_tensor=None, input_shape=None, pooling=None, classes=1000)

In [ ]:
conv_base.summary()